<a href="https://colab.research.google.com/github/VMadhav007/ML/blob/main/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install kaggle

In [17]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"madhav00769","key":"313bb256414b1c89edd5ebd93e7b5abb"}'}

In [18]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [19]:
!kaggle datasets download -d kazanova/sentiment140


Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other


In [20]:
from zipfile import ZipFile
dataset='/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('Done')

Done


nltk natural language tool


In [21]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [22]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
print(stopwords.words('english'))#these doesnt add any influential meaning to the texts!!!

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

loading data from csv to pandas dataframe

In [24]:
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO 8859-1')

In [25]:
twitter_data.shape #for checking total data

(1599999, 6)

In [26]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


Here instead of using heading its considering the 1st data set as heading Need to fix That

In [27]:
#naming the headings and reading the dataset again
column_names=['target','id','date','flag','user','text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names,encoding='ISO 8859-1')

In [28]:
twitter_data

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [29]:
#counting missing data
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [30]:
#checking the distribution of target column
twitter_data['target'].value_counts()#if the dataset doesnt have equal then we need to do upsampling and downsampling

,count
target,
0,800000
4,800000


In [31]:
#converting 4 to 1 so taht 0-neg and 1-pos
twitter_data.replace({'target':{4:1}},inplace=True)

In [32]:
twitter_data

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,1,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,1,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


Stemming - proces of reducing a word to its root word
eg:actor , actress,acting=act

In [33]:
port_stem=PorterStemmer()

In [39]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)# Remove all characters that are not letters (a-z or A-Z) and replace them with spaces
    stemmed_content = stemmed_content.lower()# Convert all characters to lowercase
    stemmed_content = stemmed_content.split()# Split the content into individual words (tokenization)
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]# For each word, if it's not a stopword (like "the", "is", "and"), apply stemming (reduce to root word)
    stemmed_content = ' '.join(stemmed_content)# Join the list of stemmed words back into a single string
    print(stemmed_content)
    return stemmed_content# Return the processed text


In [40]:
# Apply the 'stemming' function to every row in the 'text' column of the twitter_data DataFrame,
# and store the processed result in a new column called 'stemmed_content'
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

Streaming output truncated to the last 5000 lines.
whinlatt tonight forest amp zip back http tinyurl com whinlatt
usha well keep touch push updat regularli
mzpuppi bariatr fusion top tast text list use bc way
thank raevyn
neopeo bb tell
safensound hey how yeah weekend went ars lol rain ya
erinelizabethx erin darl found twitter
yuifan make last chapter lt
borntowrit isnt great kno god us sinc know weapon form shall prosper
calilewi ant learn travel twitter post
bykimbo cheer darl
chanc thank chri good see wagamama take custom servic satisfact serious nice
sangfroid correct comment
sasha miss amp love
gonna hectic next two day take today chill gonna great
lqqkout sure probabl step group help
b haha twitter bet though
anoth gorgeou day michigan c want get morn talk lay back
buckbradberri eric schmidt founder googl recruit novel take point
crime spotifi swedish compani herrey quot diggi loo diggi ley quot found anywher need golden shoe song
mwikkid cschultz yessssss second ice cream lunch


In [41]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [42]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [43]:
#separating data and label
X=twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

In [44]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [45]:
print(Y)

[0 0 0 ... 1 1 1]


NOW SPLITTING DATA to testing and train data

In [46]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [47]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


FEATURE EXTRACTION by vectoriser

In [55]:

# Create an instance of TfidfVectorizer.This converts text data into TF-IDF feature vectors (numerical representation of text)
vectorizer = TfidfVectorizer()
# Fit the vectorizer on the training data and transform it into TF-IDF vectors
# This learns the vocabulary and computes the TF-IDF weights for X_train
X_train = vectorizer.fit_transform(X_train)
# Transform the test data using the same vocabulary learned from training data
# (No fitting here to avoid data leakage)
X_test = vectorizer.transform(X_test)


In [56]:
print(X_train)


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9453092 stored elements and shape (1280000, 461488)>
  Coords	Values
  (0, 436713)	0.27259876264838384
  (0, 354543)	0.3588091611460021
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235045)	0.41996827700291095
  (0, 443066)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407301)	0.18709338684973031
  (2, 129411)	0.29074192727957143
  (2, 406399)	0.32105459490875526
  (2, 433560)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443430)	0.3348599670252845
  (2, 266729)	0.24123230668976975
  (2, 409143)	0.15169282335109835
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 288470)	0.16786949597862733
  (3, 406399)	0.29029991238662284
  (3, 158711)	0.4456939372299574
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318303)	0.21254698865277744
  (12

In [57]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2289192 stored elements and shape (320000, 461488)>
  Coords	Values
  (0, 15110)	0.1719352837797837
  (0, 31168)	0.1624772418052177
  (0, 67828)	0.26800375270827315
  (0, 106069)	0.36555450010904555
  (0, 132364)	0.255254889555786
  (0, 138164)	0.23688292264071406
  (0, 171378)	0.2805816206356074
  (0, 271016)	0.45356623916588285
  (0, 279082)	0.17825180109103442
  (0, 388348)	0.2198507607206174
  (0, 398906)	0.34910438732642673
  (0, 409143)	0.3143047059807971
  (0, 420984)	0.17915624523539805
  (1, 6463)	0.30733520460524466
  (1, 15110)	0.211037449588008
  (1, 145393)	0.575262969264869
  (1, 217562)	0.40288153995289894
  (1, 256777)	0.28751585696559306
  (1, 348135)	0.4739279595416274
  (1, 366203)	0.24595562404108307
  (2, 22532)	0.3532582957477176
  (2, 34401)	0.37916255084357414
  (2, 89448)	0.36340369428387626
  (2, 183312)	0.5892069252021465
  (2, 256834)	0.2564939661498776
  :	:
  (319994, 443794)	0.2782185641032538


Training Using logistic regression(THIS is a classifiication model)

In [58]:
model=LogisticRegression(max_iter=1000)

In [59]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

MODEL evaluation

In [60]:
#accuracy score train
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [61]:
print("Accuracy on training data :",training_data_accuracy)

Accuracy on training data : 0.79871953125


In [62]:
#accuracy score on test
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [63]:
print("Accuracy on test data :",test_data_accuracy)

Accuracy on test data : 0.77668125


MODEL Accuracy: 77.6%

SAVING MODEL

In [64]:
import pickle

In [67]:
filename ='sentiment_model.pkl'
pickle.dump(model,open(filename,'wb'))

USING model for future prediction

In [69]:
loaded_model=pickle.load(open('sentiment_model.pkl','rb'))

In [ ]:
X_new=X_test[200]
print(X_new)
print(Y_test[200])

prediction=model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print("Negative tweet")
else:
  print("positive tweet")

In [70]:
# Your custom text
my_text = "HEll nah i wouldnt give you even 1 dollar for this shit"

# Step 1: Preprocess the text using your stemming function
processed_text = stemming(my_text)
print(processed_text)

# Step 2: Convert it into TF-IDF vector using the same vectorizer
vectorized_text = vectorizer.transform([processed_text])
print(vectorized_text.toarray())


# Step 3: Use the trained model to predict
prediction = model.predict(vectorized_text)

print("Prediction:", prediction[0])


hell nah wouldnt give even dollar shit
hell nah wouldnt give even dollar shit
[[0. 0. 0. ... 0. 0. 0.]]
Prediction: 0


In [71]:
def predict(word):
  stem=stemming(word)
  vector=vectorizer.transform([stem])
  prediction = model.predict(vector)
  return "Positive" if prediction[0] == 1 else "Negative"

In [72]:
print(predict("Hell nah dawg you stupid or wot"))

hell nah dawg stupid wot
Negative
